In [ ]:
import numpy as np
import casadi as ca
import plotly.graph_objects as go

# User-defined inputs
x_lower = float(input("Enter lower bound for x: "))
x_upper = float(input("Enter upper bound for x: "))
y_lower = float(input("Enter lower bound for y: "))
y_upper = float(input("Enter upper bound for y: "))

initial_x = float(input("Enter initial guess for x: "))
initial_y = float(input("Enter initial guess for y: "))

# Validate initial guess
if not (x_lower <= initial_x <= x_upper):
    raise ValueError(f"Initial guess for x ({initial_x}) is out of bounds [{x_lower}, {x_upper}]")
if not (y_lower <= initial_y <= y_upper):
    raise ValueError(f"Initial guess for y ({initial_y}) is out of bounds [{y_lower}, {y_upper}]")

# Define the function in CasADi
x = ca.SX.sym("x")
y = ca.SX.sym("y")

# Surface building
x_rotated = 10 - x  # Rotation along a vertical axis
z = (x_rotated - 7)**2 + 1.5 * (y - 6)**2  # Minimum at (x=7, y=6)
z += ca.exp(-0.8 * ((x_rotated - 7)**2 + (y - 6)**2)) * 4

# Define the optimization problem
f = ca.Function("f", [x, y], [z])  # Define function

# Define optimization variables
opt_variables = ca.vertcat(x, y)

# Compute gradient and convert to a function
grad = ca.gradient(z, opt_variables)
grad_func = ca.Function("grad_func", [x, y], [grad])

# Armijo backtracking for step size selection
def armijo_backtracking(x_k, y_k, grad_x, grad_y, alpha=1.0, beta=0.5, sigma=1e-4):
    while f(x_k - alpha * grad_x, y_k - alpha * grad_y).full().flatten()[0] > \
          f(x_k, y_k).full().flatten()[0] - sigma * alpha * (grad_x**2 + grad_y**2):
        alpha *= beta
    return alpha

# Gradient Descent with Armijo Backtracking
x_k, y_k = initial_x, initial_y
trajectory = [(x_k, y_k, f(x_k, y_k).full().flatten()[0])]
for _ in range(50):  # Maximum iterations
    grad_vals = grad_func(x_k, y_k).full().flatten()
    step_size = armijo_backtracking(x_k, y_k, grad_vals[0], grad_vals[1])
    x_k -= step_size * grad_vals[0]
    y_k -= step_size * grad_vals[1]
    trajectory.append((x_k, y_k, f(x_k, y_k).full().flatten()[0]))
    if np.linalg.norm(grad_vals) < 1e-6:
        break

# Extract the optimization path
trajectory_x, trajectory_y, trajectory_z = zip(*trajectory)

# Generate grid data for visualization
x_values = np.linspace(x_lower, x_upper, 100)
y_values = np.linspace(y_lower, y_upper, 100)
X, Y = np.meshgrid(x_values, y_values)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = f(X[i, j], Y[i, j]).full().flatten()[0]

# Normalize Z for color display from Low to High
Z_min, Z_max = Z.min(), Z.max()
norm_Z = (Z - Z_min) / (Z_max - Z_min)

# Create an interactive 3D surface plot
fig = go.Figure(
    data=[
        go.Surface(
            x=X, y=Y, z=norm_Z,
            colorscale='Viridis',
            colorbar=dict(
                title="Magnitude",
                tickvals=[0, 1],
                ticktext=["Low", "High"],
                x=0.8  # <--- Shift colorbar to the left
            )
        )
    ]
)

# Plot the found minimum point (red dot)
fig.add_trace(go.Scatter3d(
    x=[trajectory_x[-1]],
    y=[trajectory_y[-1]],
    z=[(trajectory_z[-1] - Z_min) / (Z_max - Z_min)],
    mode='markers',
    marker=dict(size=8, color='red'),
    name='Minimizer',
    showlegend=True
))

# Plot the initial guess (blue dot)
fig.add_trace(go.Scatter3d(
    x=[initial_x],
    y=[initial_y],
    z=[(f(initial_x, initial_y).full().flatten()[0] - Z_min) / (Z_max - Z_min)],
    mode='markers',
    marker=dict(size=8, color='blue'),
    name='Initial Guess',
    showlegend=True
))

# Plot the path of the optimization
fig.add_trace(go.Scatter3d(
    x=trajectory_x,
    y=trajectory_y,
    z=[(z - Z_min) / (Z_max - Z_min) for z in trajectory_z],
    mode='lines',
    line=dict(color='black', width=2),
    name='Optimization Path',
    showlegend=True
))

# Layout 
fig.update_layout(
    title="Optimized Minimum of Rotated Asymmetric Convex Surface",
    margin=dict(l=0, r=100),  # Increase right margin so legend fits well
    scene=dict(
        xaxis_title="Reliability",
        yaxis_title="Price",
        zaxis_title="Footprint",
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False)
    ),
    legend=dict(
        x=1.02,       # Move legend further to the right
        y=1.0,        # Align to top
        xanchor='left',
        yanchor='top',
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()
